<a href="https://colab.research.google.com/github/JuanJoMontilla/Senales-y-Sistemas/blob/main/Otros/P3_EJ2Corregido2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q
!pip install pyngrok
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install streamlit yt-dlp soundfile matplotlib pandas numpy scikit-learn scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 387.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 728.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 15.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB

In [2]:
%%writefile app.py
import requests
from io import BytesIO
import subprocess
import os
import shutil
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import tempfile
from scipy.fftpack import fft
from scipy.signal import square
from scipy.interpolate import interp1d
import yt_dlp
from scipy.signal import butter, lfilter, freqz
import pandas as pd
import yt_dlp as youtube_dl
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
import joblib

# Configuración inicial de la página
st.set_page_config(page_title="Parcial 2 - Fourier SyS", page_icon="📊", layout="wide")

# Barra lateral para seleccionar el punto del parcial y mostrar información general
with st.sidebar:
    st.image("https://lh4.googleusercontent.com/proxy/WNtyuTbDjnnITJFxg1dlI63L0jfIMRf0CIKg75VavFd3ameUuokpEiXIZvafO0UbA3rGKkhjDZ2HFtRWcGiPIn7Syd37PqnCrQuXFNHguRRPYm__safRJi9Q", width=150)
    st.title("Parcial 2 - Fourier SyS")
    st.subheader("Universidad Nacional de Colombia - Sede Manizales")
    st.write("**Señales y Sistemas - 2024-1**")
    st.write("**Integrantes:**")
    st.write("- Juan Felipe Posada Niasa")
    st.write("- Juan Jose Montilla Bastidas")

    # Menú desplegable para seleccionar el punto del parcial
    punto = st.selectbox("Selecciona el punto del parcial",
                         ("Punto 1: Modulación Sistema",
                          "Punto 2: Simulación Canción YouTube"))

# Mostrar información general del punto seleccionado
if punto == "Punto 1: Expresión del Espectro de Fourier":
  st.write("### Expresión del Espectro de Fourier")

#######################################################################################
elif punto == "Punto 2: Simulación Canción YouTube":
  st.write("### Simulación Canción YouTube")
      # Función para descargar el archivo de Google Drive
  def download_file_from_drive(file_id):
      command = f"wget --load-cookies /tmp/cookies.txt 'https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id={file_id}' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id={file_id}' -O canciones.xlsx && rm -rf /tmp/cookies.txt"
      os.system(command)

  # Función para descargar audio de YouTube en formato MP3
  def download_ytvid_as_mp3(video_url, name):
      filename = f"{name}.mp3"
      options = {
          'format': 'bestaudio/best',
          'keepvideo': False,
          'outtmpl': filename,
      }
      with youtube_dl.YoutubeDL(options) as ydl:
          ydl.download([video_url])
      return filename

  # Función para convertir MP3 a WAV usando ffmpeg
  def convert_mp3_to_wav(mp3_filename, wav_filename):
      subprocess.call(['ffmpeg', '-y', '-i', mp3_filename, wav_filename])

  # Función para diseñar un filtro pasabajas
  def butter_lowpass(cutoff, fs, order=5):
      nyq = 0.5 * fs
      normal_cutoff = cutoff / nyq
      b, a = butter(order, normal_cutoff, btype='low', analog=False)
      return b, a

  def lowpass_filter(data, cutoff, fs, order=5):
      b, a = butter_lowpass(cutoff, fs, order=order)
      y = lfilter(b, a, data)
      return y, b, a

  # Función para graficar polos y ceros
  def plot_poles_zeros(b, a):
      z, p, k = tf2zpk(b, a)
      plt.figure()
      plt.scatter(np.real(z), np.imag(z), marker='o', label='Ceros', color='blue')
      plt.scatter(np.real(p), np.imag(p), marker='x', label='Polos', color='red')
      plt.axhline(0, color='black', lw=1, ls='--')
      plt.axvline(0, color='black', lw=1, ls='--')
      plt.title('Polos y Ceros')
      plt.xlabel('Parte Real')
      plt.ylabel('Parte Imaginaria')
      plt.grid()
      plt.legend()
      st.pyplot(plt)

  # Función para graficar el diagrama de Bode
  def plot_bode(b, a, fs):
      w, h = freqz(b, a, worN=8000)
      plt.figure(figsize=(10, 8))

      # Diagrama de Magnitud
      plt.subplot(2, 1, 1)
      plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h)), 'b')
      plt.title('Diagrama de Bode')
      plt.xlabel('Frecuencia [Hz]')
      plt.ylabel('Magnitud [dB]')
      plt.grid()
      plt.ylim(-100, 10)  # Ajuste de límites en dB

      # Diagrama de Fase
      plt.subplot(2, 1, 2)
      plt.plot(0.5 * fs * w / np.pi, np.angle(h), 'b')
      plt.xlabel('Frecuencia [Hz]')
      plt.ylabel('Fase [radianes]')
      plt.grid()

      plt.tight_layout()  # Asegura que los subgráficos no se superpongan
      st.pyplot(plt)

  # Interfaz de Streamlit
  st.title("Simulación de Modulación y Demodulación de Audio desde YouTube")

  # Entrada para el enlace de Google Drive
  drive_link = st.text_input("Ingresa el enlace de Google Drive de la base de canciones:")

  # Comprobar si el archivo ya existe
  if os.path.exists('canciones.xlsx'):
      st.write("La base de datos de canciones ya se ha descargado.")
      X = pd.read_excel('canciones.xlsx')
      # Limitar a un máximo de 20 canciones
      X = X.head(20)
  else:
      if st.button("Cargar Canciones"):
          # Extraer ID del enlace de Google Drive
          try:
              file_id = drive_link.split('/')[-2]  # Asegúrate de que el enlace sea válido
              download_file_from_drive(file_id)

              # Leer archivo Excel
              X = pd.read_excel('canciones.xlsx')
              # Limitar a un máximo de 20 canciones
              X = X.head(20)
              st.write(X)
          except Exception as e:
              st.error(f"Error al cargar la base de datos: {e}")

  # Entrada para el enlace de una nueva canción de YouTube
  video_url = st.text_input("Ingresa el enlace de una canción en YouTube:")

  # Parámetros ajustables con sliders
  start_time = st.slider("Tiempo de inicio del segmento (segundos)", 0, 60, 35)
  cutoff_frequency = st.slider("Frecuencia de corte del filtro (Hz)", 100, 5000, 300)

  # Procesamiento del archivo de audio si se ha ingresado un enlace
  if video_url:
      try:
          name = "audio_downloaded"
          mp3_filename = download_ytvid_as_mp3(video_url, name)
          wav_filename = f"{name}.wav"
          convert_mp3_to_wav(mp3_filename, wav_filename)

          # Leer el archivo WAV
          x, fm = sf.read(wav_filename)
          x_mono = np.mean(x, axis=1) if x.ndim > 1 else x
          ns = len(x_mono) / fm

          # Segmento seleccionado por el usuario
          end_time = start_time + 5
          start_index = int(start_time * fm)
          end_index = int(end_time * fm)

          x_mono_segment = x_mono[start_index:end_index]
          t_segment = np.arange(start_time, end_time, 1 / fm)

          # Reproducir el segmento seleccionado
          st.write(f"Reproduciendo segmento de {start_time} a {end_time} segundos.")
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_wav:
              sf.write(temp_wav.name, x_mono_segment, fm)
              st.audio(temp_wav.name)

          # Generar la señal portadora
          Ac = max(abs(x_mono_segment)) / 0.8  # Amplitud de la portadora
          c_segment = Ac * np.sin(2 * np.pi * 10000 * t_segment)  # Señal portadora

          # Señal modulada AM
          y = (1 + x_mono_segment / Ac) * c_segment

          # Graficar señal portadora y mensaje
          st.write("### Señal de Portadora y Mensaje")
          fig, ax = plt.subplots()
          ax.plot(t_segment, c_segment, label='Portadora', alpha=0.7)
          ax.plot(t_segment, x_mono_segment, label='Mensaje', alpha=0.7)
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal mensaje
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_msg:
              sf.write(temp_msg.name, x_mono_segment, fm)
              st.audio(temp_msg.name)

          # Graficar la señal modulada
          st.write("### Señal Modulada en AM")
          fig, ax = plt.subplots()
          ax.plot(t_segment, y, label='Modulada')
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal modulada
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_mod:
              sf.write(temp_mod.name, y, fm)
              st.audio(temp_mod.name)

          # Demodulación usando envolvente
          demodulated = np.abs(y)

          # Filtrado de la señal demodulada
          filtered_signal, b, a = lowpass_filter(demodulated, cutoff_frequency, fm)

          # Graficar la señal demodulada y filtrada
          st.write("### Señal Demodulada y Filtrada")
          fig, ax = plt.subplots()
          ax.plot(t_segment, filtered_signal[:len(t_segment)], label='Demodulada Filtrada')
          ax.set_xlabel('Tiempo (s)')
          ax.set_ylabel('Amplitud')
          ax.legend()
          st.pyplot(fig)

          # Reproducción de la señal demodulada filtrada
          with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_filt:
              sf.write(temp_filt.name, filtered_signal[:len(t_segment)], fm)
              st.audio(temp_filt.name)

          # Espectros de las señales
          st.write("### Espectros de las Señales")
          Xfc = np.fft.rfft(c_segment)
          Xfm = np.fft.rfft(x_mono_segment)
          Xfy = np.fft.rfft(y)
          Xfd = np.fft.rfft(filtered_signal[:len(t_segment)])
          vfre = np.fft.rfftfreq(len(c_segment), 1 / fm)

          fig, ax = plt.subplots()
          ax.plot(vfre, np.abs(Xfy), label='Modulada', alpha=0.7)
          ax.plot(vfre, np.abs(Xfm), label='Mensaje', alpha=0.7)
          ax.plot(vfre, np.abs(Xfc), label='Portadora', alpha=0.7)
          ax.plot(vfre, np.abs(Xfd), label='Filtrada', alpha=0.7)
          ax.set_xlabel('Frecuencia [Hz]')
          ax.set_ylabel('|X(f)|')
          ax.set_ylim([min(np.abs(Xfm)), 1.1 * max(np.abs(Xfm))])
          ax.legend()
          ax.grid(True)
          st.pyplot(fig)

          # Graficar polos y ceros
          plot_poles_zeros(b, a)

          # Graficar diagrama de Bode
          plot_bode(b, a, fm)

      except Exception as e:
          st.error(f"Error al procesar el audio: {e}")

Writing app.py


In [3]:
from pyngrok import ngrok

# Token personal de ngrok
token = "2lLFMBWiinIoZUi3bX7RF26eFeD_5KBTy8DqrMYoE2mf5Sqnc"
ngrok.set_auth_token(token)

# Ejecutar Streamlit en el puerto 5011
!nohup streamlit run app.py --server.port 5011 &

# Iniciar el túnel Ngrok y exponer Streamlit en una URL.
public_url = ngrok.connect(5011)
print(f"Aplicación aquí: {public_url}")

nohup: appending output to 'nohup.out'
Aplicación aquí: NgrokTunnel: "https://4e4a-34-48-62-99.ngrok-free.app" -> "http://localhost:5011"
